In [ ]:
import argparse
import logging
import requests
import csv
from io import StringIO
import sys

# Simulating command-line arguments in a notebook environment
sys.argv = ['script_name.py', '--url', 'https://raw.githubusercontent.com/Dcorpuz410/IS211_Assignment2/master/birthdays100.csv']

# Setting up the logger
def setup_logger():
    logger = logging.getLogger('assignment2')
    handler = logging.FileHandler('errors.log')
    handler.setLevel(logging.ERROR)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    return logger

# Function to download the data
def downloadData(url):
    try:
        # Ensure we're using the raw URL for GitHub files
        if not url.startswith('https://raw.githubusercontent.com'):
            raise Exception("Provided URL is not a raw GitHub URL")
        
        response = requests.get(url)
        response.raise_for_status()  # Will raise an exception for non-2xx status codes
        return response.text
    except requests.exceptions.RequestException as e:
        raise Exception(f"Error downloading data: {e}")

# Function to process the CSV data into a dictionary
def processData(csvData):
    personData = {}
    csv_reader = csv.reader(StringIO(csvData))
    for row in csv_reader:
        if row:  # Check if the row is not empty
            try:
                id = int(row[0])
                name = row[1]
                birthday = row[2]
                personData[id] = (name, birthday)
            except ValueError as e:
                print(f"Skipping row due to error: {e}")
    return personData

# Function to display a person's details
def displayPerson(personData, id):
    if id in personData:
        name, birthday = personData[id]
        print(f"ID: {id} - Name: {name} - Birthday: {birthday}")
    else:
        print(f"ID {id} not found.")

# Main function
def main():
    # Set up argument parsing
    parser = argparse.ArgumentParser(description="Download and process birthday data.")
    parser.add_argument('--url', required=True, help="URL of the CSV file")
    args = parser.parse_args()

    # Set up logger
    logger = setup_logger()

    # Initialize personData to None, so it's clear if it fails to initialize
    personData = None

    # Download and process the CSV data
    try:
        csvData = downloadData(args.url)
    except Exception as e:
        logger.error(str(e))
        print(f"Error: {e}")
        exit(1)

    try:
        personData = processData(csvData)
    except Exception as e:
        logger.error(f"Error processing data: {e}")
        print(f"Error processing data: {e}")
        exit(1)

    if personData is None:
        print("Error: Failed to process the data.")
        exit(1)

    # Keep asking for user input until they enter a number <= 0
    while True:
        try:
            user_id = int(input("Enter an ID to lookup (or <= 0 to exit): "))
            if user_id <= 0:
                print("Exiting the program.")
                break
            displayPerson(personData, user_id)
        except ValueError:
            print("Invalid input. Please enter a valid number.")

if __name__ == "__main__":
    main()

# To run the script, use the following command:
# python script_name.py --url "https://raw.githubusercontent.com/Dcorpuz410/IS211_Assignment2/master/birthdays100.csv"


Skipping row due to error: invalid literal for int() with base 10: 'id'


Enter an ID to lookup (or <= 0 to exit):  5


ID: 5 - Name: Gavin Ball - Birthday: 20/03/1983


Enter an ID to lookup (or <= 0 to exit):  16


ID: 16 - Name: Felicity Churchill - Birthday: 09/11/1983


Enter an ID to lookup (or <= 0 to exit):  21


ID: 21 - Name: Dorothy Lee - Birthday: 16/03/1999
